 ### The  `task`  and  `taskwait`  Constructs 

 The following example shows how to traverse a tree-like structure using explicit  tasks. Note that the  `traverse`  function should be called from within a  parallel region for the different specified tasks to be executed in parallel. Also  note that the tasks will be executed in no specified order because there are no  synchronization directives. Thus, assuming that the traversal will be done in post  order, as in the sequential code, is wrong. 

In [ ]:
%load ../sources/Example_tasking.1.c 

In [ ]:
%load ../sources/Example_tasking.1.f90 

 In the next example, we force a postorder traversal of the tree by adding a  `taskwait`   directive. Now, we can safely assume that the left and right sons have been executed  before we process the current node. 

In [ ]:
%load ../sources/Example_tasking.2.c 

In [ ]:
%load ../sources/Example_tasking.2.f90 

 The following example demonstrates how to use the  `task`  construct to process  elements of a linked list in parallel. The thread executing the  `single`   region generates all of the explicit tasks, which are then executed by the threads  in the current team. The pointer  _p_  is  `firstprivate`  by default  on the  `task`  construct so it is not necessary to specify it in a  `firstprivate`   clause. 

In [ ]:
%load ../sources/Example_tasking.3.c 

In [ ]:
%load ../sources/Example_tasking.3.f90 

 The  `fib()`  function should be called from within a  `parallel`   region  for the different specified tasks to be executed in parallel. Also, only one thread  of the  `parallel`  region should call  `fib()`  unless multiple concurrent  Fibonacci computations are desired.  

In [ ]:
%load ../sources/Example_tasking.4.c 

In [ ]:
%load ../sources/Example_tasking.4.f 

 Note: There are more efficient algorithms for computing Fibonacci numbers. This  classic recursion algorithm is for illustrative purposes. 

 The following example demonstrates a way to generate a large number of tasks with  one thread and execute them with the threads in the team. While generating these  tasks, the implementation may reach its limit on unassigned tasks.  If it does,  the implementation is allowed to cause the thread executing the task generating  loop to suspend its task at the task scheduling point in the  `task`  directive,  and start executing unassigned tasks.  Once the number of unassigned tasks is sufficiently  low, the thread may resume execution of the task generating loop. 

In [ ]:
%load ../sources/Example_tasking.5.c 

In [ ]:
%load ../sources/Example_tasking.5.f 

 The following example is the same as the previous one, except that the tasks are  generated in an untied task. While generating the tasks, the implementation may  reach its limit on unassigned tasks. If it does, the implementation is allowed  to cause the thread executing the task generating loop to suspend its task at the  task scheduling point in the  `task`  directive, and start executing unassigned  tasks.  If that thread begins execution of a task that takes a long time to complete,  the other threads may complete all the other tasks before it is finished. 

 In this case, since the loop is in an untied task, any other thread is eligible  to resume the task generating loop. In the previous examples, the other threads  would be forced to idle until the generating thread finishes its long task, since  the task generating loop was in a tied task. 

In [ ]:
%load ../sources/Example_tasking.6.c 

In [ ]:
%load ../sources/Example_tasking.6.f 

 The following two examples demonstrate how the scheduling rules illustrated in  Section 2.11.3 of the OpenMP 4.0 specification affect the usage of   `threadprivate`  variables in tasks. A  `threadprivate`   variable can be modified by another task that is executed by the same thread. Thus,  the value of a  `threadprivate`  variable cannot be assumed to be unchanged  across a task scheduling point. In untied tasks, task scheduling points may be  added in any place by the implementation. 

 A task switch may occur at a task scheduling point. A single thread may execute  both of the task regions that modify  `tp` . The parts of these task regions  in which  `tp`  is modified may be executed in any order so the resulting  value of  `var`  can be either 1 or 2. 

In [ ]:
%load ../sources/Example_tasking.7.c 

In [ ]:
%load ../sources/Example_tasking.7.f 

 In this example, scheduling constraints prohibit a thread in the team from executing  a new task that modifies  `tp`   while another such task region tied to the  same thread is suspended. Therefore, the value written will persist across the  task scheduling point. 

In [ ]:
%load ../sources/Example_tasking.8.c 

In [ ]:
%load ../sources/Example_tasking.8.f 

 The following two examples demonstrate how the scheduling rules illustrated in  Section 2.11.3 of the OpenMP 4.0 specification affect the usage of locks  and critical sections in tasks.  If a lock is held  across a task scheduling point, no attempt should be made to acquire the same lock  in any code that may be interleaved.  Otherwise, a deadlock is possible. 

 In the example below, suppose the thread executing task 1 defers task 2.  When  it encounters the task scheduling point at task 3, it could suspend task 1 and  begin task 2 which will result in a deadlock when it tries to enter critical region  1. 

In [ ]:
%load ../sources/Example_tasking.9.c 

In [ ]:
%load ../sources/Example_tasking.9.f 

 In the following example,  `lock`  is held across a task scheduling point.   However, according to the scheduling restrictions, the executing thread can't  begin executing one of the non-descendant tasks that also acquires  `lock`  before  the task region is complete.  Therefore, no deadlock is possible. 

In [ ]:
%load ../sources/Example_tasking.10.c 

In [ ]:
%load ../sources/Example_tasking.10.f90 

 The following examples illustrate the use of the  `mergeable`  clause in the   `task`  construct. In this first example, the  `task`  construct has  been annotated with the  `mergeable`   clause. The addition of this clause  allows the implementation to reuse the data environment (including the ICVs) of  the parent task for the task inside  `foo`  if the task is included or undeferred.  Thus, the result of the execution may differ depending on whether the task is merged  or not. Therefore the mergeable clause needs to be used with caution. In this example,  the use of the mergeable clause is safe. As  `x`  is a shared variable the  outcome does not depend on whether or not the task is merged (that is, the task  will always increment the same variable and will always compute the same value  for  `x` ). 

In [ ]:
%load ../sources/Example_tasking.11.c 

In [ ]:
%load ../sources/Example_tasking.11.f90 

 This second example shows an incorrect use of the  `mergeable`  clause. In  this example, the created task will access different instances of the variable   `x`  if the task is not merged, as  `x`  is  `firstprivate` , but  it will access the same variable  `x`  if the task is merged. As a result,  the behavior of the program is unspecified and it can print two different values  for  `x`  depending on the decisions taken by the implementation. 

In [ ]:
%load ../sources/Example_tasking.12.c 

In [ ]:
%load ../sources/Example_tasking.12.f90 

 The following example shows the use of the  `final`  clause and the  `omp_in_final`   API call in a recursive binary search program. To reduce overhead, once a certain  depth of recursion is reached the program uses the  `final`  clause to create  only included tasks, which allow additional optimizations. 

 The use of the  `omp_in_final`  API call allows programmers to optimize  their code by specifying which parts of the program are not necessary when a task  can create only included tasks (that is, the code is inside a  `final`  task).  In this example, the use of a different state variable is not necessary so once  the program reaches the part of the computation that is finalized and copying from  the parent state to the new state is eliminated. The allocation of  `new_state`   in the stack could also be avoided but it would make this example less clear. The   `final`  clause is most effective when used in conjunction with the  `mergeable`   clause since all tasks created in a  `final`  task region are included tasks  that can be merged if the  `mergeable`  clause is present. 

In [ ]:
%load ../sources/Example_tasking.13.c 

In [ ]:
%load ../sources/Example_tasking.13.f90 

 The following example illustrates the difference between the  `if`   and the   `final`  clauses. The  `if`  clause has a local effect. In the first  nest of tasks, the one that has the  `if`   clause will be undeferred but  the task nested inside that task will not be affected by the  `if`  clause  and will be created as usual. Alternatively, the  `final`  clause affects  all  `task`  constructs in the  `final`  task region but not the  `final`   task itself. In the second nest of tasks, the nested tasks will be created as included  tasks. Note also that the conditions for the  `if`  and  `final`  clauses  are usually the opposite. 

In [ ]:
%load ../sources/Example_tasking.14.c 

In [ ]:
%load ../sources/Example_tasking.14.f90 

---end--- 